# Forte Tutorial 1.03: Forte's support for handling sparse states and operators

Forte exposes several functions to create and manipulate general second quantized operators and wave functions.
In this tutorial we will look at simple examples that illustrate how these classes work.

## Preliminaries
Here we define a useful function to display equations in LaTeX format

In [41]:
import math
import forte
from IPython.display import display, Math, Latex
from forte import det, SparseState, SparseOperator

def latex(obj):
    """Call the latex() function on an object and display the returned value in LaTeX"""
    display(Math(obj.latex()))

In a previous tutorial we looked at how to define determinants in forte. Here we are going to use the utility function `forte.det()`, which creates a determinant from a string representation of the determinant. The occupation of each orbital is specified by the symbols `2` (doubly occupied), `+` (single alpha electron), `-` (single beta electron), `0` (empty).

Here are some examples.

In [ ]:
print(det('22+-'))
print(det('22ba'))
print(det('ABBA'))

Depending on the size if the `Determinant` class, these commands will return a 64 bit or longer representation of the determinants.

## The SparseState class

Sparse collections of determinants can be manipulated using the `SparseState` class. The simplest way to create a `SparseState` object is by passing a dictionary of `determinants -> double`. For example, here we create a superposition of a determinant with two electrons and one that has no electrons, both with equal coefficients normalized to one
$$
|\Psi\rangle = \frac{1}{\sqrt{2}}\left( |20\rangle + |00\rangle \right)
$$

In [ ]:
c = 1./ math.sqrt(2.0)
psi = SparseState({ forte.det('20'): c, forte.det('00') : c})
print(psi)

An alternative way to print this wave function is by calling the `str` method on the `SparseState` object. The argument `2` here indicates that we want to show only the occupation numbers of only the first two orbitals.

In [ ]:
print(psi.str(2))

## The `SparseOperator` class

The `SparseOperator` class can handle operators of the form
$$
\hat{O} = \sum_{pqrs\cdots} t_{pq\cdots}^{rs\cdots} \hat{a}^\dagger_p \hat{a}^\dagger_q \cdots \hat{a}_s \hat{a}_r
$$
where each individual term in the summation can be an arbitrary order operator.
However, the amplitudes are assumed to be **real numbers**.

### Creating `SparseOperator` objects

After creation, a `SparseOperator` object is empty
```python
op = forte.SparseOperator()
latex(op)
# displays nothing
```

The simplest way to populate a `SparseOperator` is by adding one term at a time using the `add` function.

A generic operator
$$
\hat{q}_1 \hat{q}_2 \cdots, \quad \text{ with } \hat{q}_i \in \{ \hat{a}_p, \hat{a}^\dagger_p\}
$$
can be specified using the following syntax
```
add('[<orbital_1><spin_1><type_1> <orbital_2><spin_2><type_2> ...]', amplitude)
```
where
```
orbital_i: int
spin_i: 'a' (alpha) or 'b' (beta)
type_i: '+' (creation) or '-' (annihilation)
```

For example, the operator $-2 \hat{a}^\dagger_{1_\alpha} \hat{a}^\dagger_{2_\beta} \hat{a}_{2_\beta} \hat{a}_{0_\alpha}$ is encoded as `[1a+ 2b+ 3b- 0a-]` and can be generated with the following code:

In [ ]:
op = forte.SparseOperator()
op.add('[1a+ 2b+ 3b- 0a-]',-2.0)
latex(op)

We can also directly build an operator using the `make_sparse_operator` convenience fuction, which uses the same syntax of `add_term_from_str` but allows to pass one or more operators as a list of pairs. For example, if we want to build the operator $\frac{1}{2} (\hat{a}_{0_\alpha} - \hat{a}^\dagger_{0_\alpha})$ we can do the following:

In [ ]:
op = forte.sparse_operator([('[0a-]',0.5),('[0a+]',-0.5)])
latex(op)

Note that `SparseOperator` objects are addressable by operator string:

In [ ]:
print(f"{op['[0a-]'] = }")
print(f"{op['[0a+]'] = }")
print(f"{op['[]'] = }")

### Ordering of second quantized operators in the `SparseOperator` object

<div class="alert alert-block alert-warning">
Note that `add_term_from_str` <b>assumes that the operators will match a specific order!</b>

This canonical order is defined as
$$
(\alpha \text{ creation}) (\beta \text{ creation}) (\beta \text{ annihilation}) (\alpha \text{ annihilation})
$$
with the creation (annihilation) operators ordered within each group in increasing (decreasing) order.
The following operator satisfies the canonical order:
$$
+\;\hat{a}_{2 \alpha}^\dagger\hat{a}_{3 \alpha}^\dagger\hat{a}_{2 \beta}^\dagger\hat{a}_{3 \beta}^\dagger\hat{a}_{1 \beta}\hat{a}_{0 \beta}\hat{a}_{1 \alpha}\hat{a}_{0 \alpha}
$$
</div>

If you want to work with operators that do not follow this ordering, for example, $\hat{a}_{1 \alpha}\hat{a}^\dagger_{0 \alpha}$, you will need to work out an equivalent representation, for example, $\hat{a}_{0 \alpha}\hat{a}^\dagger_{0 \alpha} = 1 - \hat{a}^\dagger_{0 \alpha}\hat{a}_{0 \alpha}$.

These examples illustrate valid operators in canonical order

In [ ]:
# beta annihilation operators appear to the left of alpha annihilation
# within each group, orbital indices decrease going from left to right
op = forte.sparse_operator('[1b- 0b- 1a- 0a-]',1.0)
latex(op)

# beta creation operators appear to the right of alpha annihilation
# within each group, orbitals increase going from left to right
op = forte.sparse_operator('[0a+ 1a+ 0b+ 1b+]',1.0)
latex(op)

# creation operators appear to the left of annihilation operators
op = forte.sparse_operator('[2a+ 3a+ 2b+ 3b+ 1b- 0b- 1a- 0a-]',1.0)
latex(op)

When the operator passed is out of order, an exception is thrown. For example, the following code
```python
op = forte.sparse_operator('[0b- 1b- 1a- 0a-]',1.0)
latex(op)
```
leads to the following RuntimeError

In [ ]:
try:
    op = forte.sparse_operator('[0b- 1b- 1a- 0a-]',1.0)
except Exception as e:
    print(f'RuntimeError: {e}')

This error can be overriden. However, **this is recommended only if you understand what happens when you do so**. The function `add_term_from_str` has an extra option that allows it to reorder the operators to the canonical form. The final operator is multiplied by a sign factor that corresponds to the parity of the permutation that connects the initial and final ordering. This code illustrates how this reordering happens

In [ ]:
# the operators [0a- 0b- 1a- 1b-] are reordered and the final sign is -1. 
op = forte.sparse_operator('[0a- 0b- 1a- 1b-]',1.0,allow_reordering=True)
latex(op)

# the operators [0a- 0b- 1a- 1b-] are reordered and the final sign is -1. 
op = forte.sparse_operator('[0a- 0b- 1a- 1b-]',1.0,allow_reordering=True)
latex(op)

# The operator [0a- 0b- 1a- 1b-] (see above) is equivalent to -[1a- 1b- 0b- 0a-].
op = forte.sparse_operator('[1a- 1b- 0b- 0a-]',-1.0,allow_reordering=True)
latex(op)

# Another example that illustrates the reordering of operators
op = forte.sparse_operator('[0a- 0b- 1a- 1b- 2a+ 2b+ 3a+ 3b+]',1.0,allow_reordering=True)
latex(op)

An exception is also thrown if two operators are repeated. For example, the following code
```python
op = forte.SparseOperator()
op = forte.make_sparse_operator('[0b- 0b-]',1.0)
```
gives to the following RuntimeError

In [ ]:
try:
    op = forte.sparse_operator('[0b- 0b-]',1.0)
except Exception as e:
    print(f'RuntimeError: {e}')

### Specifying a full operator with the `SparseOperator` class

To form a full operator we can just keep adding terms to a `SparseOperator` object or directly create a `SpareOperator` object by passing a list of pairs (string,coefficient) to the function `make_sparse_operator`. For example:

In [ ]:
op1 = forte.SparseOperator()
op1.add('[1a+ 0a-]',0.3)
op1.add('[1b+ 0b-]',0.3)
op1.add('[1a+ 1b+ 0b- 0a-]',0.1)
latex(op1)

op2 = forte.sparse_operator([('[1a+ 0a-]',0.3),
                            ('[1b+ 0b-]',0.3),
                            ('[1a+ 1b+ 0b- 0a-]',0.1)])
latex(op2)

assert op1 == op2

## Applying a `SparseOperator` to a `SparseState`

To apply an operator to a state vector you can use the `forte.apply_operator(op,psi)` function. This function takes an operator (`op`) and a state (`psi`), and returns the state `|new_psi> = op |psi>`. For example, the following creates a CIS wave function using the operator
$$
\hat{T} = 0.1\; +0.3 \left(\hat{a}_{1 \alpha}^\dagger\hat{a}_{0 \alpha} + \hat{a}_{1 \beta}^\dagger\hat{a}_{0 \beta} \right)
$$
where the first term is just a scalar

In [ ]:
op = forte.SparseOperator()
op.add('[]',0.1)
op.add('[1a+ 0a-]',0.3)
op.add('[1b+ 0b-]',0.3)
psi = forte.SparseState({ forte.det('2'): 1.0})
new_psi = forte.apply_op(op,psi)
print(new_psi.str(3))

# test the apply_op function
assert new_psi[forte.det('2')] == 0.1
assert new_psi[forte.det('-+')] == 0.3
assert new_psi[forte.det('+-')] == 0.3

### Exponential operator

To apply the exponential operator $\exp(\hat{T})$ we can use the class `SparseExp` class. This class provides the method `apply_op` which takes as arguments the operator and the state

In [ ]:
psi = forte.SparseState({ forte.det('2'): 1.0})
exp_op = forte.SparseExp()
new_psi = exp_op.apply_op(op,psi)
print(new_psi.str(3))

assert abs(new_psi[forte.det('200')]-1.105171) < 1e-6
assert abs(new_psi[forte.det('020')]-0.099465) < 1e-6
assert abs(new_psi[forte.det('+-0')]-0.331551) < 1e-6
assert abs(new_psi[forte.det('-+0')]-0.331551) < 1e-6

There are several variables that control the behavior of `apply_op`. For example, to compute the inverse, we can just apply $\exp(-\hat{T})$

In [ ]:
new_psi2 = exp_op.apply_op(op,new_psi,scaling_factor=-1.0)
print(new_psi2.str(3))

We can also control other parameters, like the order of the Taylor expansion used to approximate $\exp(\hat{T})$ (`maxk`) and a threshold used to screen term (`screen_thresh`). For example, to apply $1 + \hat{T}$ we can call

In [ ]:
psi = forte.SparseState({ forte.det('2'): 1.0})
exp_op = forte.SparseExp(maxk=1)
new_psi = exp_op.apply_op(op,psi)
print(new_psi.str(3))

### Exponential of an anti-Hermitian operator

The exponential function can also be used to compute the action of the anti-Hermitian operator $\hat{T} - \hat{T}^\dagger$. This operation is implemented in the class `SparseExp` via the method `apply_antiherm` which takes as arguments the operator and the state. Here is a simple example:

In [ ]:
psi = forte.SparseState({ forte.det('2'): 1.0})
exp_op = forte.SparseExp()
new_psi = exp_op.apply_antiherm(op,psi)
print(new_psi.str(3))
norm = forte.overlap(new_psi,new_psi)
assert abs(norm-1.0) < 1e-6
assert abs(new_psi[forte.det('200')]-0.912668) < 1e-6
assert abs(new_psi[forte.det('020')]-0.087332) < 1e-6
assert abs(new_psi[forte.det('+-0')]-0.282321) < 1e-6
assert abs(new_psi[forte.det('-+0')]-0.282321) < 1e-6

### Factorized exponential of an anti-Hermitian operator

Another useful operator is the factorized exponential. Given a **list** of operators
$$
(t_1 \hat{\kappa}_1, t_2 \hat{\tau}_2, \ldots,t_N \hat{\tau}_N)
$$
the factorized exponential is defined as
$$
\prod_\mu^N \exp(t_\mu \hat{\tau}_\mu) = \exp(t_N \hat{\tau}_N) \cdots \exp(t_1 \hat{\tau}_1)  
$$
This operation is implemented in the class `SparseFactExp`.
This class provides the method `apply_antiherm` which takes as arguments the operator and the state. Here is a simple example:

In [ ]:
op = forte.SparseOperatorList()
op.add('[1a+ 0a-]',0.3)
op.add('[1b+ 0b-]',0.3)

psi = forte.SparseState({ forte.det('2'): 1.0})
factexp_op = forte.SparseFactExp()
new_psi = factexp_op.apply_antiherm(op,psi)
print(new_psi.str(3))

# test the apply_antiherm function
assert abs(new_psi[forte.det('200')]-0.912668) < 1e-6
assert abs(new_psi[forte.det('020')]-0.087332) < 1e-6
assert abs(new_psi[forte.det('+-0')]-0.282321) < 1e-6
assert abs(new_psi[forte.det('-+0')]-0.282321) < 1e-6

To compute the inverse of the factorized exponential, just pass the option `inverse=True` to `apply_antiherm`:

In [ ]:
starting_psi = factexp_op.apply_antiherm(op,new_psi,inverse=True)
print(starting_psi.str(3))

### Similarity transformations of operators (New!)

Forte can also perform similarity transformations of Hamiltonians (see [arXiv:2408.09636](https://arxiv.org/abs/2408.09636)). For example, the factorized unitary transformation of a `SparseOperator` $O$ by a sequence of exponentials:
$$
\bar{O} = \cdots \exp(-t_2 \hat{\tau}_2) \exp(-t_1 \hat{\tau}_1)  O \exp(t_1 \hat{\tau}_1) \exp(t_2 \hat{\tau}_2) \cdots  
$$
can be evaluated with the function `fact_unitary_trans_antiherm`

In [ ]:
O = forte.sparse_operator('[1a+ 0a-]',1.0)
print(f'Untransformed operator = {O}')
T = forte.operator_list([('[2a+ 0a-]',0.3),('[1a+ 0a-]',0.5)])
print(f'Transformation generator = {T}')

In [ ]:
Obar = O.fact_unitary_trans_antiherm(T)
print(f'Transformed operator = {Obar}')

assert Obar['[1a+ 2a-]'] == -0.2593433800522308
assert Obar['[1a+ 0a-]'] == 0.735753498540072

It is also possible to evaluate the gradient of a transformation:
$$
\frac{\partial}{\partial t_k} \bar{O} = \frac{\partial}{\partial t_k} \cdots \exp(-t_2 \hat{\tau}_2) \exp(-t_1 \hat{\tau}_1)  O \exp(t_1 \hat{\tau}_1) \exp(t_2 \hat{\tau}_2) \cdots  
$$
This quantity can be evaluated with the function `fact_unitary_trans_antiherm_grad`


In [ ]:
Obar_grad = O.fact_unitary_trans_antiherm_grad(T,0)
print(f'Transformed operator = {Obar_grad}')

assert Obar_grad['[0a+ 0a-]'] == -0.12433583966497525
assert Obar_grad['[0a+ 1a-]'] == 0.0679249787857943

And we can easily verify that the computed gradients agree with numerical ones:

In [ ]:
h = 1e-6
for i in range(2):
    Obar_grad = O.fact_unitary_trans_antiherm_grad(T,i)
    print(f'Analytical gradient = {Obar_grad}')

    # Two-point symmetric finite difference gradient
    Tp = forte.SparseOperatorList(T)
    Tp[i] = T[i] + h
    Tm = forte.SparseOperatorList(T)
    Tm[i] = T[i] - h
    Obar_grad_num = (O.fact_unitary_trans_antiherm(Tp) - O.fact_unitary_trans_antiherm(Tm))/(2 * h)
    print(f'Numerical gradient = {Obar_grad_num}')

    print(f'Difference norm = {(Obar_grad - Obar_grad_num).norm()}')
    assert (Obar_grad - Obar_grad_num).norm() < 1e-6